In [175]:
import numpy as np
import pandas as pd
import sqlite3 as sql
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import a_funciones as fn
from mlxtend.preprocessing import TransactionEncoder

# <h1> Preprocesamiento y exploracion

In [176]:
conn=sql.connect('db_movies') ### crear cuando no existe el nombre de cd  y para conectarse cuando sí existe.
cur=conn.cursor() ###para funciones que ejecutan sql en base de datos

In [177]:
# Imprimir las tablas existentes en db_movies
cur.execute("SELECT name FROM sqlite_master where type='table' ")
cur.fetchall()

[('ratings',), ('movies',)]

In [178]:
# Se observa que tiene cada tabla 
movies= pd.read_sql("""select *  from movies""", conn)
movies_ratings = pd.read_sql('select * from ratings', conn)

In [181]:
# Se verifica el tipo de datos y número de nulos 
movies.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


In [182]:
# verificar duplicados 
movies.duplicated().sum() 

0

In [183]:
# Se verifica el tipo de datos y número de nulos 
movies_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [184]:
# verificar duplicados 
movies_ratings.duplicated().sum()

0

<h2> Exploración ratings

In [185]:
cr=pd.read_sql(""" select 
                          rating, 
                          count(*) as conteo 
                          from ratings
                          group by rating
                          order by conteo desc""", conn)
cr
                          

,rating,conteo
0,4.0,26818
1,3.0,20047
2,5.0,13211
3,3.5,13136
4,4.5,8551
5,2.0,7551
6,2.5,5550
7,1.0,2811
8,1.5,1791
9,0.5,1370


In [186]:
# Se imprime histograma con los ratings para verificar su distribución
data  = go.Bar( x=cr.rating,y=cr.conteo, text=cr.conteo, textposition="outside")
Layout=go.Layout(title="Conteo de ratings",xaxis={'title':'Rating'},yaxis={'title':'Conteo'})
go.Figure(data,Layout)

In [187]:
#Se realiza la exploración de los nulos
cr=pd.read_sql(""" select count(*) as contador_nulo
                          from ratings
                          where rating is null """, conn)
cr


,contador_nulo
0,0


<h2> Filtro usuario segun numero de calificaciones

In [188]:
# Usuarios que calificaron las películas y el número de calificaciones de cada uno

rating_users=pd.read_sql(''' select userId,
                         count(*) as cnt_rat
                         from ratings
                         group by userId
                         order by cnt_rat desc
                         ''',conn )
rating_users

,userId,cnt_rat
0,414,2698
1,599,2478
2,474,2108
3,448,1864
4,274,1346
...,...,...
605,207,20
606,194,20
607,189,20
608,147,20


In [189]:
# Se realiza el histograma de frecuencia del número de calificaciones por usuario
fig  = px.histogram(rating_users, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show() 

In [190]:
# Verificar datos principales de la distribución actual
rating_users.describe()

,userId,cnt_rat
count,610.000000,610.000000
mean,305.500000,165.304918
std,176.236111,269.480584
min,1.000000,20.000000
25%,153.250000,35.000000
50%,305.500000,70.500000
75%,457.750000,168.000000
max,610.000000,2698.000000


In [191]:
# Se aplica un filtro para los usuarios que tienen entre 20 y 800 calificaciones
rating_users2=pd.read_sql(''' select userId,
                         count(*) as cnt_rat
                         from ratings
                         group by userId
                         having cnt_rat >=20 and cnt_rat <=800
                         order by cnt_rat desc
                         ''',conn)
rating_users2

,userId,cnt_rat
0,600,763
1,590,728
2,483,728
3,105,722
4,19,703
...,...,...
584,207,20
585,194,20
586,189,20
587,147,20


In [192]:
# Se realiza el histograma de frecuencia del número de calificaciones por usuario con el filtro aplicado 
fig  = px.histogram(rating_users2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones por usario')
fig.show() 

In [193]:
# Verificar datos principales de la distribución con filtro
rating_users2.describe()

,userId,cnt_rat
count,589.000000,589.00000
mean,302.806452,125.93039
std,176.223574,141.26513
min,1.000000,20.00000
25%,149.000000,35.00000
50%,303.000000,66.00000
75%,456.000000,154.00000
max,609.000000,763.00000


<h2> Filtro películas según número de visualizaciones

In [195]:
# Se imprime la tabla movies y ratings juntas
df=pd.read_sql("""select * from consolidacion
               """, conn)
df

,movieId,title,genres,userId,movieId:1,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,1,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,1,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,1,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,1,4.5,1305696483
...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184,193581,4.0,1537109082
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184,193583,3.5,1537109545
100833,193585,Flint (2017),Drama,184,193585,3.5,1537109805
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184,193587,3.5,1537110021


In [196]:
# Verificar cuantas calificaciones tiene cada película
rating_movies=pd.read_sql(''' select title,
                         count(*) as cnt_rat
                         from consolidacion
                         group by title
                         order by cnt_rat desc
                         ''',conn )
rating_movies

,title,cnt_rat
0,Forrest Gump (1994),329
1,"Shawshank Redemption, The (1994)",317
2,Pulp Fiction (1994),307
3,"Silence of the Lambs, The (1991)",279
4,"Matrix, The (1999)",278
...,...,...
9714,'night Mother (1986),1
9715,'Tis the Season for Love (2015),1
9716,'Salem's Lot (2004),1
9717,'Hellboy': The Seeds of Creation (2004),1


In [197]:
# Se realiza el histograma de frecuencia del número de calificaciones por película
fig  = px.histogram(rating_movies, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada película')
fig.show()  

In [198]:
# Verificar datos principales de la distribución
rating_movies.describe()

,cnt_rat
count,9719.000000
mean,10.375141
std,22.406220
min,1.000000
25%,1.000000
50%,3.000000
75%,9.000000
max,329.000000


In [199]:
# Se aplica un filtro para las películas que tienen más de 5 calificaciones
rating_movies2=pd.read_sql(''' select title ,
                         count(*) as cnt_rat
                         from consolidacion
                         group by title
                         having cnt_rat>=5
                         order by cnt_rat desc
                         ''',conn )
rating_movies2


,title,cnt_rat
0,Forrest Gump (1994),329
1,"Shawshank Redemption, The (1994)",317
2,Pulp Fiction (1994),307
3,"Silence of the Lambs, The (1991)",279
4,"Matrix, The (1999)",278
...,...,...
3646,"Accused, The (1988)",5
3647,"Accidental Tourist, The (1988)",5
3648,"7th Voyage of Sinbad, The (1958)",5
3649,54 (1998),5


In [200]:
# Se realiza el histograma de frecuencia del número de calificaciones por película con filtro
fig  = px.histogram(rating_movies2, x= 'cnt_rat', title= 'Hist frecuencia de numero de calificaciones para cada libro')
fig.show()  

In [201]:
# Verificar datos principales de la distribución
rating_movies2.describe()

,cnt_rat
count,3651.000000
mean,24.728294
std,31.701339
min,5.000000
25%,7.000000
50%,13.000000
75%,28.000000
max,329.000000


In [202]:
# Ejecutar los preprocesamientos realizados en el .sql
fn.ejecutar_sql('preprocesamientosql.sql', cur)

In [203]:
# Verificar las bases con las que quedaron 
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()


[('ratings',),
 ('movies',),
 ('usuarios_sel',),
 ('consolidacion',),
 ('movies_sel',),
 ('ratings_final',),
 ('base_lista2',),
 ('filtro3',),
 ('base_lista3',),
 ('filtro_rat',),
 ('filtro_rat2',),
 ('consolidacion2',)]

In [204]:
# Imprimir tabla movies
cr=pd.read_sql(""" select * from movies
                          """, conn)
cr

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


# <h3> Separación de géneros 

In [205]:
genres=cr['genres'].str.split('|')
genres

0       [Adventure, Animation, Children, Comedy, Fantasy]
1                          [Adventure, Children, Fantasy]
2                                       [Comedy, Romance]
3                                [Comedy, Drama, Romance]
4                                                [Comedy]
                              ...                        
9737                 [Action, Animation, Comedy, Fantasy]
9738                         [Animation, Comedy, Fantasy]
9739                                              [Drama]
9740                                  [Action, Animation]
9741                                             [Comedy]
Name: genres, Length: 9742, dtype: object

In [206]:
#Separarlo en un Dataframe
te=TransactionEncoder()
genres= te.fit_transform(genres)
genres= pd.DataFrame(genres, columns=te.columns_)

genres

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,False,False,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
1,False,False,True,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False
3,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,False,True,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
9738,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False
9739,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
9740,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [227]:
# Convertir todo a enteros 
genres=genres.astype(int)
genres.head(3)

,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,0,0,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0


# <h3> Concatenación

In [209]:
#Concatenación de Genres con Cr 

resultado = pd.concat([cr, genres],axis=1)
movies2 = resultado.drop(["genres"],axis=1)
movies2.head()

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [210]:
# Verificación de la información
movies2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             9742 non-null   int64 
 1   title               9742 non-null   object
 2   (no genres listed)  9742 non-null   int32 
 3   Action              9742 non-null   int32 
 4   Adventure           9742 non-null   int32 
 5   Animation           9742 non-null   int32 
 6   Children            9742 non-null   int32 
 7   Comedy              9742 non-null   int32 
 8   Crime               9742 non-null   int32 
 9   Documentary         9742 non-null   int32 
 10  Drama               9742 non-null   int32 
 11  Fantasy             9742 non-null   int32 
 12  Film-Noir           9742 non-null   int32 
 13  Horror              9742 non-null   int32 
 14  IMAX                9742 non-null   int32 
 15  Musical             9742 non-null   int32 
 16  Mystery             9742

In [211]:
# Se exporta la base a db_movies
movies2.to_sql('movies2', conn, if_exists= 'replace')

9742

<h2> Extracción año de lanzamiento para base_lista3_a

In [212]:
cr=pd.read_sql(""" select * from consolidacion2
                          """, conn)
cr

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller
...,...,...,...,...,...
64625,609,786,3.0,Eraser (1996),Action|Drama|Thriller
64626,609,828,3.0,"Adventures of Pinocchio, The (1996)",Adventure|Children
64627,609,892,3.0,Twelfth Night (1996),Comedy|Drama|Romance
64628,609,1056,3.0,Jude (1996),Drama


In [213]:
# Conecta a la base de datos (cambia esto a tu base de datos real)
conn = sql.connect('db_movies')

# Ejecuta la consulta SQL sin intentar extraer el año
query = """
SELECT title, rating
FROM consolidacion2;
"""
result = pd.read_sql(query, conn)


# Función para extraer el año de un título usando una expresión regular en Python
import re

def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return match.group(1)
    else:
        return None

# Aplica la función de extracción de año a la columna 'title'
result['year'] = result['title'].apply(extract_year)

# Filtra los registros con años válidos
result = result[result['year'].notna()]

# Convierte la columna 'year' a tipo numérico (entero)
result['year'] = result['year'].astype(int)

In [214]:
# Se imprime resultados de los años que se publicó la película
result=result[['year']]
result

,year
0,1995
1,1995
2,1995
3,1995
4,1995
...,...
64625,1996
64626,1996
64627,1996
64628,1996


In [215]:
# Se concatenan las bases de consolidación con los años en que fueron estrenadas.
resultado = pd.concat([cr, result],axis=1)

base_lista3_a=resultado

In [216]:
# Se exporta la base a db_movies
base_lista3_a.to_sql('base_lista3_a', conn, if_exists= 'replace')

64630

<h2> Extracción año de lanzamiento para movies 2

In [228]:
cr=pd.read_sql(""" select * from movies2
                          """, conn)
cr.head(3)

,level_0,index,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,0,0,1,Toy Story (1995),0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,1995.0
1,1,1,2,Jumanji (1995),0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1995.0
2,2,2,3,Grumpier Old Men (1995),0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,1995.0


In [218]:
# Conecta a la base de datos (cambia esto a tu base de datos real)
conn = sql.connect('db_movies')

# Ejecuta la consulta SQL sin intentar extraer el año
query = """
SELECT title
FROM movies2;
"""
result = pd.read_sql(query, conn)


# Función para extraer el año de un título usando una expresión regular en Python
import re

def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return match.group(1)
    else:
        return None

# Aplica la función de extracción de año a la columna 'title'
result['year'] = result['title'].apply(extract_year)

# Filtra los registros con años válidos
result = result[result['year'].notna()]

# Convierte la columna 'year' a tipo numérico (entero)
result['year'] = result['year'].astype(int)

In [219]:
# Se imprime resultados de los años que se publicó la película
result=result[['year']]
result

,year
0,1995
1,1995
2,1995
3,1995
4,1995
...,...
9737,2017
9738,2017
9739,2017
9740,2018


In [220]:
# Se concatenan las bases de consolidación con los años en que fueron estrenadas.
resultado = pd.concat([cr, result],axis=1)

movies2=resultado

In [221]:
#Se borran los datos nulos después de agregar la fecha.
movies2=movies2.dropna()

In [222]:
# Se exporta la base a db_movies
movies2.to_sql('movies2', conn, if_exists= 'replace')

9729

<h2> Extraccion año que fue vista

In [223]:

df=pd.read_sql("""select strftime('%Y', datetime(timestamp, 'unixepoch')) as date
            from base_lista3
            """, conn)
df


,date
0,2009
1,2000
2,2005
3,2003
4,2009
...,...
64625,2011
64626,2008
64627,2001
64628,2017


In [224]:
# Se concatena la base con el año.
base_lista3_a=pd.concat([base_lista3_a, df],axis=1)

In [226]:
# Se exporta la base a db_movies
base_lista3_a.to_sql('base_lista3_a', conn, if_exists= 'replace')

64630